### Section 1

Initializing the Libraries into notebook

In [ ]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore') #ignoring some deprication warnings

Importing the raw data and checking for missing values

In [15]:
df = pd.read_csv('crop_yield.csv')

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Region                  1000000 non-null  object 
 1   Soil_Type               1000000 non-null  object 
 2   Crop                    1000000 non-null  object 
 3   Rainfall_mm             1000000 non-null  float64
 4   Temperature_Celsius     1000000 non-null  float64
 5   Fertilizer_Used         1000000 non-null  bool   
 6   Irrigation_Used         1000000 non-null  bool   
 7   Weather_Condition       1000000 non-null  object 
 8   Days_to_Harvest         1000000 non-null  int64  
 9   Yield_tons_per_hectare  1000000 non-null  float64
dtypes: bool(2), float64(3), int64(1), object(4)
memory usage: 62.9+ MB


### Check for NaN Values

In [14]:
print("Number of NaN values in each column:")
print(df.isna().sum())

Number of NaN values in each column:
Region                    0
Soil_Type                 0
Crop                      0
Rainfall_mm               0
Temperature_Celsius       0
Fertilizer_Used           0
Irrigation_Used           0
Weather_Condition         0
Days_to_Harvest           0
Yield_tons_per_hectare    0
dtype: int64


### Pre-Processing required. 
**Encoding**
* Region
* Soil Type
* Crop
* Weather Condition

**Scaling**
* Rainfall_mm
* Temperature_Celsius 
* Days_to_Harvest
* Yield_tons_per_hectare

In [ ]:
#Code area for pre-processing i guess